In [ ]:
import pandas as pd
import numpy as np

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
try:
    orders_df = pd.read_csv('orders.csv')
    items_df = pd.read_csv('items.csv')
    products_df = pd.read_csv('products.csv')
except FileNotFoundError as e:
    print(f"Error: File not found. Check name: {e}")
    print("SIMULATED DATA is used to demonstrate logic.")

    N_ORDERS = 10000
    N_PRODUCTS = 50
    N_ITEMS = 74818
    np.random.seed(42)

    orders_df = pd.DataFrame({
        'Order ID': np.arange(1, N_ORDERS + 1),
        'Order Datetime': pd.to_datetime('2024-01-01 08:00:00') +
                          pd.to_timedelta(np.arange(N_ORDERS) * np.random.randint(1, 10), unit='m'),
        'Order Price': np.random.uniform(5.0, 150.0, N_ORDERS).round(2)
    })

    products_df = pd.DataFrame({
        'Product ID': np.arange(1, N_PRODUCTS + 1),
        'Product Name': [f'Product {i}' for i in range(1, N_PRODUCTS + 1)],
        'Product Price': np.random.uniform(5.0, 50.0, N_PRODUCTS).round(2)
    })

    items_df = pd.DataFrame({
        'Order ID': np.random.choice(orders_df['Order ID'], N_ITEMS),
        'Product ID': np.random.choice(products_df['Product ID'], N_ITEMS),
        'Quantity': np.random.randint(1, 6, N_ITEMS),
    })

# Об'єднуємо таблиці
df = items_df.merge(
    products_df[['Product ID', 'Product Name', 'Product Price']],
    on='Product ID', how='left'
).merge(
    orders_df[['Order ID', 'Order Datetime', 'Order Price']],
    on='Order ID', how='left'
)

df.to_csv('combined_restaurant_data.csv', index=False)

print(f"Number of rows in final dataframe: {len(df)}")
print(f"Expected number of rows: 74818. Match: {len(df) == 74818}")
print("\nFirst 5 rows of merged dataframe:")
print(df.head(), "\n")


# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
quantity_counts = df.groupby('Product Name')['Quantity'].sum().sort_values(ascending=False)
top_10_quantity = quantity_counts.head(10)

total_quantity = quantity_counts.sum()
others_quantity_count = total_quantity - top_10_quantity.sum()

pie_data_quantity = pd.concat([
    top_10_quantity.rename("Count"),
    pd.Series(others_quantity_count, index=['OTHERS']).rename("Count")
], axis=0).to_frame()

pie_data_quantity['Percentage (%)'] = (pie_data_quantity['Count'] / total_quantity * 100).round(2)
pie_data_quantity['Count'] = pie_data_quantity['Count'].astype(int)

print("### Task 1: Top 10 most popular products (by QUANTITY) ###")
print(pie_data_quantity, "\n")


# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
df['Item Price'] = df['Product Price'] * df['Quantity']

revenue_by_product = df.groupby('Product Name')['Item Price'].sum().sort_values(ascending=False)
top_10_revenue = revenue_by_product.head(10)

total_revenue = revenue_by_product.sum()
others_revenue_count = total_revenue - top_10_revenue.sum()

pie_data_revenue = pd.concat([
    top_10_revenue.rename("Revenue"),
    pd.Series(others_revenue_count, index=['OTHERS']).rename("Revenue")
], axis=0).to_frame()

pie_data_revenue['Percentage (%)'] = (pie_data_revenue['Revenue'] / total_revenue * 100).round(2)
pie_data_revenue['Revenue'] = pie_data_revenue['Revenue'].round(2)

print("### Task 2: Top 10 Most Profitable Products (by REVENUE) ###")
print(pie_data_revenue, "\n")


# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
orders_df['Order Datetime'] = pd.to_datetime(orders_df['Order Datetime'])
orders_df['Order Hour'] = orders_df['Order Datetime'].dt.hour

hourly_income_total = orders_df.groupby('Order Hour')['Order Price'].sum().round(2)
full_hours = pd.Series(0.0, index=range(24))
hourly_income_total = hourly_income_total.combine_first(full_hours).sort_index().rename('Total Income')

print("### Task 3: Total Hourly Income (Order Hour 0-23) ###")
print(hourly_income_total.to_frame(), "\n")

# ASCII bar chart
def create_bar_chart_ascii(series, title) -> str:
    max_val = series.max()
    output = [f"--- {title} ---"]
    for index, value in series.items():
        bar_length = int((value / max_val) * 40) if max_val != 0 else 0
        bar = "#" * bar_length
        output.append(f"Hour {index:02d}: | {bar:<40} {value:,.2f}")
    return "\n".join(output)

print(create_bar_chart_ascii(hourly_income_total, "Hourly Income"), "\n")


# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

orders_df['Order Day Of The Week'] = orders_df['Order Datetime'].dt.day_name().astype(
    pd.CategoricalDtype(categories=day_order, ordered=True)
)

daily_income_total = orders_df.groupby('Order Day Of The Week')['Order Price'].sum().round(2).sort_index().rename('Total Income')

print("### Task 4: Total Income by Day of the Week ###")
print(daily_income_total.to_frame(), "\n")

print(create_bar_chart_ascii(daily_income_total, "Income by Day of the Week"))
